In [1]:
%gui tk
from netgen import gui
#from netgen.geom2d import unit_square
from netgen import geom2d
from netgen.meshing import *

In [2]:
# global mesh
geo = geom2d.SplineGeometry()
p1,p2,p3,p4 = [ geo.AppendPoint(x,y) for x,y in [(0,0), (1,0), (1,1), (0,1)] ]
geo.Append (["line", p1, p2], leftdomain=1, rightdomain=0)
geo.Append (["line", p2, p3], leftdomain=1, rightdomain=0)
geo.Append (["line", p3, p4], leftdomain=1, rightdomain=0)
geo.Append (["line", p4, p1], leftdomain=1, rightdomain=0)


In [7]:
msh = geo.GenerateMesh(maxh=.2)

In [3]:
# map each vertex to a list of its adjacent elements
def vmap(msh):
    vs = {}
    for e in msh.Elements2D():
        for v in e.vertices:
            es = vs.get(v.nr)
            if es:
                es.append(e)
            else:
                vs[v.nr] = [e]
    return vs   

# return a set of the unique vertices for each vertex patch
def vsets(vmap):
    vsets_dict = {}
    for v in vmap.keys():
        vset = set([v1 for e in vmap[v] for v1 in e.vertices])
        vsets_dict[v] = vset
    return vsets_dict


In [4]:
# takes a list of points representing the boundary of the vertex patch
# returns a geometry
def add_bnd_to_geom(bnd, geo):
    ps = [geo.AppendPoint(*v) for v in bnd]
    for i in range(len(ps)-1):
        geo.Append (["line", ps[i],ps[i+1]], leftdomain=2, rightdomain = 1)
    geo.Append(["line", ps[-1],ps[0]], leftdomain=2, rightdomain = 1)


# takes a list of 'elements' where each 'element' is an ordered list of vertex nrs
# the first n-1 elements are triangles, the last is a polygon
# returns an ordered list of boundary vertices (polygon)
def make_boundary(els, center):
    if len(els) == 0: raise ArgumentError("make_boundary")
    if len(els) == 1: return els[0]
    poly = els.pop()
    e, v1, v2, rest = find_common(poly, els)
    if v1 == None:
        poly.remove(center)
        return poly
    
    merged = merge_two(poly,e,v1,v2)
    rest.append(merged)
    return make_boundary(rest, center)


# takes a polygon and a list of at least two triangles 
# and extracts the first triangle found which shares an edge with the polygon
def find_common(poly, els):
    e, v1, v2 = find_match(poly, els)
    els.remove(e)
    return e, v1, v2, els

# takes a polygon and a list of triangles, at least one of which has an edge in common with the given polygon
# returns a matching triangle along with the two common vertices
def find_match(poly, els):
    for e in els:
        if e[0] in poly and e[1] in poly and e[2] in poly:
            return e, None, None
        if e[0] in poly and e[1] in poly:
            return e, e[0],e[1]
        if e[0] in poly and e[2] in poly:
            return e, e[0],e[2]
        if e[1] in poly and e[2] in poly:
            return e, e[1],e[2]
    raise ArgumentError("find_match")
    
# given two elements and their common vertices, 
# return a merged element representing the boundary of the pair
def merge_two(poly, e, v1, v2):
    i1 = poly.index(v1)
    i2 = poly.index(v2)
    e.remove(v1)
    e.remove(v2)
    if i2-i1 == 1: return poly[:i1+1]+e+poly[i2:]
    elif i1-i2 == 1: return poly[:i2+1]+e+poly[i1:]
    else: return poly + e

In [5]:
# generate dict mapping vertices to adjoining elements
vm = vmap(msh)
# generate dict mapping each vertex to the set of vertices in the patch
vs = vsets(vm)
# the current vertex number
v = 35

# elements for the current vertex
els = vm[v]

# vertex set for the current vertex
vset = vs[v]
# ordered list of points in the global mesh
pts = [p for p in msh.Points()]

# first make a list of sublists, each of which contains the vertex indices of one element
vs = [[v.nr for v in el.vertices] for el in els]
vs

[[34, 37, 35],
 [32, 34, 35],
 [26, 27, 35],
 [27, 32, 35],
 [26, 35, 31],
 [31, 35, 36],
 [35, 37, 36]]

In [6]:
# construct a boundary polygon from this list
bndnums = make_boundary(vs, v)
# map the vertex numbers of the polygon to 3D points
bndpts = [pts[v - 1] for v in bndnums]
vs,bndnums
# make the local geometry
add_bnd_to_geom(bndpts, geo)
msh.SetGeometry(geo)
